In [ ]:
#참고 : https://bigdata-doctrine.tistory.com/34

In [54]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from fake_useragent import UserAgent
import datetime
from urllib import parse
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd

URL = "https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1={}&sid2={}&date={}&page={}"
SID = [
    {'sid1': '100', 'sid2': '264', 'name1': '정치', 'name2': '대통령실'}, 
    {'sid1': '100', 'sid2': '265', 'name1': '정치', 'name2': '국회/정당'}, 
    {'sid1': '100', 'sid2': '268', 'name1': '정치', 'name2': '북한'}, 
    {'sid1': '100', 'sid2': '266', 'name1': '정치', 'name2': '행정'}, 
    {'sid1': '100', 'sid2': '267', 'name1': '정치', 'name2': '국방/외교'}, 
    {'sid1': '100', 'sid2': '269', 'name1': '정치', 'name2': '정치일반'}, 
    {'sid1': '101', 'sid2': '259', 'name1': '경제', 'name2': '금융'}, 
    {'sid1': '101', 'sid2': '258', 'name1': '경제', 'name2': '증권'}, 
    {'sid1': '101', 'sid2': '261', 'name1': '경제', 'name2': '산업/재계'}, 
    {'sid1': '101', 'sid2': '771', 'name1': '경제', 'name2': '중기/벤처'}, 
    {'sid1': '101', 'sid2': '260', 'name1': '경제', 'name2': '부동산'}, 
    {'sid1': '101', 'sid2': '262', 'name1': '경제', 'name2': '글로벌 경제'}, 
    {'sid1': '101', 'sid2': '310', 'name1': '경제', 'name2': '생활경제'}, 
    {'sid1': '101', 'sid2': '263', 'name1': '경제', 'name2': '경제 일반'}, 
    {'sid1': '102', 'sid2': '249', 'name1': '사회', 'name2': '사건사고'}, 
    {'sid1': '102', 'sid2': '250', 'name1': '사회', 'name2': '교육'}, 
    {'sid1': '102', 'sid2': '251', 'name1': '사회', 'name2': '노동'}, 
    {'sid1': '102', 'sid2': '254', 'name1': '사회', 'name2': '언론'}, 
    {'sid1': '102', 'sid2': '252', 'name1': '사회', 'name2': '환경'}, 
    {'sid1': '102', 'sid2': '59b', 'name1': '사회', 'name2': '인권/복지'}, 
    {'sid1': '102', 'sid2': '255', 'name1': '사회', 'name2': '식품/의료'}, 
    {'sid1': '102', 'sid2': '256', 'name1': '사회', 'name2': '지역'}, 
    {'sid1': '102', 'sid2': '276', 'name1': '사회', 'name2': '인물'}, 
    {'sid1': '102', 'sid2': '257', 'name1': '사회', 'name2': '사회 일반'}, 
    {'sid1': '103', 'sid2': '241', 'name1': '생활문화', 'name2': '건강정보'}, 
    {'sid1': '103', 'sid2': '239', 'name1': '생활문화', 'name2': '자동차/시승기'}, 
    {'sid1': '103', 'sid2': '240', 'name1': '생활문화', 'name2': '도로/교통'}, 
    {'sid1': '103', 'sid2': '237', 'name1': '생활문화', 'name2': '여행/레저'}, 
    {'sid1': '103', 'sid2': '238', 'name1': '생활문화', 'name2': '음식/맛집'}, 
    {'sid1': '103', 'sid2': '376', 'name1': '생활문화', 'name2': '패션/뷰티'}, 
    {'sid1': '103', 'sid2': '242', 'name1': '생활문화', 'name2': '공연/전시'}, 
    {'sid1': '103', 'sid2': '243', 'name1': '생활문화', 'name2': '책'}, 
    {'sid1': '103', 'sid2': '244', 'name1': '생활문화', 'name2': '종교'}, 
    {'sid1': '103', 'sid2': '248', 'name1': '생활문화', 'name2': '날씨'}, 
    {'sid1': '103', 'sid2': '245', 'name1': '생활문화', 'name2': '생활문화 일반'}, 
    {'sid1': '104', 'sid2': '231', 'name1': '세계', 'name2': '아시아/호주'}, 
    {'sid1': '104', 'sid2': '232', 'name1': '세계', 'name2': '미국/중남미'}, 
    {'sid1': '104', 'sid2': '233', 'name1': '세계', 'name2': '유럽'}, 
    {'sid1': '104', 'sid2': '234', 'name1': '세계', 'name2': '중동/아프리카'}, 
    {'sid1': '104', 'sid2': '322', 'name1': '세계', 'name2': '세계 일반'}, 
    {'sid1': '105', 'sid2': '731', 'name1': 'IT/과학', 'name2': '모바일'}, 
    {'sid1': '105', 'sid2': '226', 'name1': 'IT/과학', 'name2': '인터넷/SNS'}, 
    {'sid1': '105', 'sid2': '227', 'name1': 'IT/과학', 'name2': '통신/뉴미디어'}, 
    {'sid1': '105', 'sid2': '230', 'name1': 'IT/과학', 'name2': 'IT 일반'}, 
    {'sid1': '105', 'sid2': '732', 'name1': 'IT/과학', 'name2': '보안/해킹'}, 
    {'sid1': '105', 'sid2': '283', 'name1': 'IT/과학', 'name2': '컴퓨터'}, 
    {'sid1': '105', 'sid2': '229', 'name1': 'IT/과학', 'name2': '게임/리뷰'}, 
    {'sid1': '105', 'sid2': '228', 'name1': 'IT/과학', 'name2': '과학 일반'},
]
newlines = {
    "br", "div", "p"
}
DAY = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - datetime.timedelta(days=1)
DAY = DAY.strftime('%Y%m%d')
UA = UserAgent().random

def Crawling(SID, URL, DAY, UA):
    url_list = []
    for sid in SID:
        url = URL.format(sid['sid1'], sid['sid2'], DAY, '10')
        url_list.append(url)
    return url
        
def art_crawl(url):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        urls(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    # art_dic = {}
    
    ## 1.
    # title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    # url = urls[sid][index]
    # url = urls['url']
    html = requests.get(url, headers={"User-Agent":  "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0"}, verify=False)
    soup = BeautifulSoup(html.text, "lxml")
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    return date_str, main_str


""" 뉴스 페이지 목록 조회 """
def news_list_page(date):
    keys = set()
    urls = []
    sid1set = []
    sid2set = []
    dataset_set = {}
    # 뉴스테마별 딕셔너리
    urlset = []
    for sid in SID:
        #urlset[sid["sid1"], sid["sid2"]] = []
        # max page ?
        for page in range(1,999):
            sid1, sid2, page = sid["sid1"], sid["sid2"], page
            key = f"{sid1}.{sid2}.{page}"
            if key not in keys:
                keys.add(key)
                sid1set.append(sid1)
                sid2set.append(sid2)
                urlset.append(URL.format(sid1, sid2, date, page))
    dataset_set['sid1'] = sid1set
    dataset_set['sid2'] = sid2set
    dataset_set['link'] = urlset
    
    dataset_sid1 = []
    dataset_sid2 = []
    dataset_link = []
    
    DF_set = pd.DataFrame(dataset_set)

    for i,j in tqdm(DF_set.groupby(['sid1','sid2'])[['link']]):
        sid_main = []
        sid_sub = []
        url_main = []
        break_key = []
        print(i)
        # 각 세부테마별 중복 url 존재시 멈춤 (ex. 마지막 페이지 다음 페이지에서는 url이 겹치므로 정지)
        for sid1, sid2, url in zip(j['sid1'],j['sid2'],j['link']):
            html = requests.get(url, headers={"User-Agent":  "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0"}, verify=False)
            soup = BeautifulSoup(html.text, "lxml")
            #name1, name2 = names[index][0], names[index][1]       
            """ 뉴스 페이지 목록 조회 """
            items = soup.select("#main_content > div.list_body.newsflash_body > ul > li > dl > dt[class!='photo'] > a")
            for item in items:
                url, title = item["href"], item.text.strip()  
                if url not in url_main:
                    if 0 < len(title):   
                        sid_main.append(sid1)
                        sid_sub.append(sid2)
                        url_main.append(url)
                else:
                    # 메인섹션 url 수집 중단 key
                    break_key.append(1)
                    break
            if len(break_key) == 0:
                if len(url_main) != 0:
                    dl = [x for x in url_main if x not in dataset_link]
                    dataset_link.extend(dl)
                    #print(len(dataset_link))
                    dataset_sid1.extend(sid_main[:len(dl)])
                    dataset_sid2.extend(sid_sub[:len(dl)])
            else:
                break
        print("=============================================수집완료======================================================")
        
    return dataset_sid1, dataset_sid2, dataset_link

In [55]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

dum1, dum2, dum3 = news_list_page('20230918')

  0%|          | 0/48 [00:00<?, ?it/s]

('100', '264')
=============================================수집완료======================================================
('100', '265')
=============================================수집완료======================================================
('100', '266')
=============================================수집완료======================================================
('100', '267')
=============================================수집완료======================================================
('100', '268')
=============================================수집완료======================================================
('100', '269')
=============================================수집완료======================================================
('101', '258')
=============================================수집완료======================================================
('101', '259')
=============================================수집완료======================================================
('101', '260')
=================================

In [56]:
len(dum1),len(dum2),len(dum3)

(23476, 23476, 23476)

In [57]:
df = pd.DataFrame([x for x in zip(dum1, dum2, dum3)], columns = ['sid1','sid2','url'])
df

,sid1,sid2,url
0,100,264,https://n.news.naver.com/mnews/article/001/001...
1,100,264,https://n.news.naver.com/mnews/article/079/000...
2,100,264,https://n.news.naver.com/mnews/article/421/000...
3,100,264,https://n.news.naver.com/mnews/article/001/001...
4,100,264,https://n.news.naver.com/mnews/article/001/001...
...,...,...,...
23471,105,732,https://n.news.naver.com/mnews/article/031/000...
23472,105,732,https://n.news.naver.com/mnews/article/138/000...
23473,105,732,https://n.news.naver.com/mnews/article/014/000...
23474,105,732,https://n.news.naver.com/mnews/article/138/000...


In [58]:
def art_crawl(url):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

dataset = []    
for i,j in tqdm(df.iterrows()):
    art_dic = art_crawl(j['url'])
    if 0 < len(art_dic["title"]):
        dataset.append({"sid1": j['sid1'], "sid2": j['sid2'], "url": j['url'], "title": art_dic["title"], "date": art_dic["date"], "main": art_dic["main"]})

0it [00:00, ?it/s]

In [59]:
import pandas as pd

art_df = pd.DataFrame(dataset)

In [60]:
art_df

,sid1,sid2,url,title,date,main
0,100,264,https://n.news.naver.com/mnews/article/001/001...,"바이든 ""이란 억류 미국인 석방 도운 한국 정부 등에 감사""",2023.09.18. 오후 11:50,바이든 미국 대통령[AFP 연합뉴스 자료사진. 재판매 및 DB 금지](워싱턴=연합뉴...
1,100,264,https://n.news.naver.com/mnews/article/079/000...,"尹, 유엔 총회 참석차 미국 뉴욕 도착…4박6일 일정 시작",2023.09.18. 오후 11:42,제78차 유엔총회에 참석하는 윤석열 대통령과 부인 김건희 여사가 18일(현지시간) ...
2,100,264,https://n.news.naver.com/mnews/article/421/000...,"윤 대통령, 미국 뉴욕 도착…유엔 총회 일정 시작(종합)",2023.09.18. 오후 11:37,JFK 국제공항 착륙…4박6일 일정 수행주유엔 대사·주뉴욕 총영사 나와 맞이\n\n...
3,100,264,https://n.news.naver.com/mnews/article/001/001...,"尹대통령, 뉴욕 도착…엑스포 유치 총력전(종합)",2023.09.18. 오후 11:36,"美 방문 첫날에만 10개국 가까운 나라 정상들과 만남\n\n\n\n윤 대통령, 유엔..."
4,100,264,https://n.news.naver.com/mnews/article/001/001...,"""투표용지서 트럼프 이름 빼도 되는지 법원 판단 받아보자""",2023.09.18. 오후 11:34,"캘리포니아 주의회 의원들, 州법무장관에 서한…수정헌법 14조 해석 차원'내란 선동'..."
...,...,...,...,...,...,...
23313,105,732,https://n.news.naver.com/mnews/article/031/000...,"과기정통부, '사이버안전센터' 세종청사로 이전 개소",2023.09.18. 오후 1:48,외부 사이버 공격 대응 위한 24시간 운영 전문가 조직과학기술정보통신부는 18일 '...
23314,105,732,https://n.news.naver.com/mnews/article/138/000...,"과기정통부, 사이버안전센터 나주에서 세종으로 이전",2023.09.18. 오전 10:31,ⓒ과기정통부[디지털데일리 이종현기자] 과학기술정보통신부(이하 과기정통부)는 과학기술...
23315,105,732,https://n.news.naver.com/mnews/article/014/000...,"라온시큐어, 다음달 17일 '시큐업 세미나 2023' 개최",2023.09.18. 오전 10:25,라온시큐어는 디지털 보안·인증 정보 공유 세미나 ‘시큐업(Security IQ UP...
23316,105,732,https://n.news.naver.com/mnews/article/138/000...,"[DD's톡] 신한투자증권, 지니언스 목표 주가 1만8000원… “성장 원년될 것”",2023.09.18. 오전 10:14,ⓒ지니언스[디지털데일리 이종현기자] 신한투자증권이 18일 사이버보안 산업에 대한 보...


In [61]:
# 출력 옵션 설정
# pd.set_option("display.max_rows", None)  # 모든 행 표시
pd.set_option("display.max_columns", None)  # 모든 열 표시
pd.set_option("display.width", None)  # 줄 바꿈 없이 전체 내용 표시

In [62]:
# 텍스트 전처리
import re
def text_preprocessing(text):
    text = re.sub('[\n,\t]', '', text)
    text = text.replace('   ', ' ')
    text = text.replace('\\', '')
    return text

In [63]:
art_df['main'] = art_df['main'].apply(text_preprocessing)

In [64]:
from datetime import datetime

def to_dt_time(date):
    if date != '':
        date = datetime.strptime(date[:10], '%Y.%m.%d')
        date = date.strftime('%Y-%m-%d')
        return date
    else:
        return date

In [65]:
# 날짜
now = datetime.now()
now = now.date().strftime('%Y-%m-%d')

art_df['ymd'] = art_df['date'].apply(to_dt_time)

In [66]:
art_df

,sid1,sid2,url,title,date,main,ymd
0,100,264,https://n.news.naver.com/mnews/article/001/001...,"바이든 ""이란 억류 미국인 석방 도운 한국 정부 등에 감사""",2023.09.18. 오후 11:50,바이든 미국 대통령[AFP 연합뉴스 자료사진. 재판매 및 DB 금지](워싱턴=연합뉴...,2023-09-18
1,100,264,https://n.news.naver.com/mnews/article/079/000...,"尹, 유엔 총회 참석차 미국 뉴욕 도착…4박6일 일정 시작",2023.09.18. 오후 11:42,제78차 유엔총회에 참석하는 윤석열 대통령과 부인 김건희 여사가 18일(현지시간) ...,2023-09-18
2,100,264,https://n.news.naver.com/mnews/article/421/000...,"윤 대통령, 미국 뉴욕 도착…유엔 총회 일정 시작(종합)",2023.09.18. 오후 11:37,JFK 국제공항 착륙…4박6일 일정 수행주유엔 대사·주뉴욕 총영사 나와 맞이윤석열 ...,2023-09-18
3,100,264,https://n.news.naver.com/mnews/article/001/001...,"尹대통령, 뉴욕 도착…엑스포 유치 총력전(종합)",2023.09.18. 오후 11:36,美 방문 첫날에만 10개국 가까운 나라 정상들과 만남윤 대통령 유엔총회 참석 위해 ...,2023-09-18
4,100,264,https://n.news.naver.com/mnews/article/001/001...,"""투표용지서 트럼프 이름 빼도 되는지 법원 판단 받아보자""",2023.09.18. 오후 11:34,캘리포니아 주의회 의원들 州법무장관에 서한…수정헌법 14조 해석 차원'내란 선동' ...,2023-09-18
...,...,...,...,...,...,...,...
23313,105,732,https://n.news.naver.com/mnews/article/031/000...,"과기정통부, '사이버안전센터' 세종청사로 이전 개소",2023.09.18. 오후 1:48,외부 사이버 공격 대응 위한 24시간 운영 전문가 조직과학기술정보통신부는 18일 '...,2023-09-18
23314,105,732,https://n.news.naver.com/mnews/article/138/000...,"과기정통부, 사이버안전센터 나주에서 세종으로 이전",2023.09.18. 오전 10:31,ⓒ과기정통부[디지털데일리 이종현기자] 과학기술정보통신부(이하 과기정통부)는 과학기술...,2023-09-18
23315,105,732,https://n.news.naver.com/mnews/article/014/000...,"라온시큐어, 다음달 17일 '시큐업 세미나 2023' 개최",2023.09.18. 오전 10:25,라온시큐어는 디지털 보안·인증 정보 공유 세미나 ‘시큐업(Security IQ UP...,2023-09-18
23316,105,732,https://n.news.naver.com/mnews/article/138/000...,"[DD's톡] 신한투자증권, 지니언스 목표 주가 1만8000원… “성장 원년될 것”",2023.09.18. 오전 10:14,ⓒ지니언스[디지털데일리 이종현기자] 신한투자증권이 18일 사이버보안 산업에 대한 보...,2023-09-18


In [67]:
SID[0]['sid1']

'100'

In [68]:
SID_df = pd.DataFrame(SID)

In [69]:
df_tot = pd.merge(art_df, SID_df, on = ['sid1', 'sid2'], how = 'inner')

In [70]:
df_tot

,sid1,sid2,url,title,date,main,ymd,name1,name2
0,100,264,https://n.news.naver.com/mnews/article/001/001...,"바이든 ""이란 억류 미국인 석방 도운 한국 정부 등에 감사""",2023.09.18. 오후 11:50,바이든 미국 대통령[AFP 연합뉴스 자료사진. 재판매 및 DB 금지](워싱턴=연합뉴...,2023-09-18,정치,대통령실
1,100,264,https://n.news.naver.com/mnews/article/079/000...,"尹, 유엔 총회 참석차 미국 뉴욕 도착…4박6일 일정 시작",2023.09.18. 오후 11:42,제78차 유엔총회에 참석하는 윤석열 대통령과 부인 김건희 여사가 18일(현지시간) ...,2023-09-18,정치,대통령실
2,100,264,https://n.news.naver.com/mnews/article/421/000...,"윤 대통령, 미국 뉴욕 도착…유엔 총회 일정 시작(종합)",2023.09.18. 오후 11:37,JFK 국제공항 착륙…4박6일 일정 수행주유엔 대사·주뉴욕 총영사 나와 맞이윤석열 ...,2023-09-18,정치,대통령실
3,100,264,https://n.news.naver.com/mnews/article/001/001...,"尹대통령, 뉴욕 도착…엑스포 유치 총력전(종합)",2023.09.18. 오후 11:36,美 방문 첫날에만 10개국 가까운 나라 정상들과 만남윤 대통령 유엔총회 참석 위해 ...,2023-09-18,정치,대통령실
4,100,264,https://n.news.naver.com/mnews/article/001/001...,"""투표용지서 트럼프 이름 빼도 되는지 법원 판단 받아보자""",2023.09.18. 오후 11:34,캘리포니아 주의회 의원들 州법무장관에 서한…수정헌법 14조 해석 차원'내란 선동' ...,2023-09-18,정치,대통령실
...,...,...,...,...,...,...,...,...,...
23313,105,732,https://n.news.naver.com/mnews/article/031/000...,"과기정통부, '사이버안전센터' 세종청사로 이전 개소",2023.09.18. 오후 1:48,외부 사이버 공격 대응 위한 24시간 운영 전문가 조직과학기술정보통신부는 18일 '...,2023-09-18,IT/과학,보안/해킹
23314,105,732,https://n.news.naver.com/mnews/article/138/000...,"과기정통부, 사이버안전센터 나주에서 세종으로 이전",2023.09.18. 오전 10:31,ⓒ과기정통부[디지털데일리 이종현기자] 과학기술정보통신부(이하 과기정통부)는 과학기술...,2023-09-18,IT/과학,보안/해킹
23315,105,732,https://n.news.naver.com/mnews/article/014/000...,"라온시큐어, 다음달 17일 '시큐업 세미나 2023' 개최",2023.09.18. 오전 10:25,라온시큐어는 디지털 보안·인증 정보 공유 세미나 ‘시큐업(Security IQ UP...,2023-09-18,IT/과학,보안/해킹
23316,105,732,https://n.news.naver.com/mnews/article/138/000...,"[DD's톡] 신한투자증권, 지니언스 목표 주가 1만8000원… “성장 원년될 것”",2023.09.18. 오전 10:14,ⓒ지니언스[디지털데일리 이종현기자] 신한투자증권이 18일 사이버보안 산업에 대한 보...,2023-09-18,IT/과학,보안/해킹


In [71]:
df_tot[['name1', 'name2','url']].groupby(['name1', 'name2']).count()

url
name1 name2        
IT/과학 IT 일반     492
      게임/리뷰      81
      과학 일반     142
      모바일        29
      보안/해킹      17
      인터넷/SNS   135
      컴퓨터        46
      통신/뉴미디어   159
경제    경제 일반    2092
      글로벌 경제     90
      금융        469
      부동산       418
      산업/재계    1651
      생활경제      413
      중기/벤처     155
      증권        848
사회    교육        360
      노동        149
      사건사고      815
      사회 일반    2936
      식품/의료     156
      언론         38
      인권/복지      43
      인물        196
      지역       4881
      환경        142
생활문화  건강정보      125
      공연/전시     185
      날씨        124
      도로/교통      20
      생활문화 일반   735
      여행/레저     170
      음식/맛집      25
      자동차/시승기   150
      종교         67
      책          55
      패션/뷰티      34
세계    미국/중남미    209
      세계 일반     494
      아시아/호주    269
      유럽        148
      중동/아프리카    57
정치    국방/외교     255
      국회/정당    1299
      대통령실      239
      북한        185
      정치일반     1296
      행정        224

In [72]:
df_tot.to_csv("df_tot.csv")